In [1]:
import config
#DB instance creation
import mysql.connector
from mysql.connector import errorcode
import requests


In [2]:
def connect_to_db():
    cnx = mysql.connector.connect(
    host=config.host, user=config.user, passwd=config.password, database=config.DB_NAME)
    cursor = cnx.cursor()
    return cnx, cursor


In [3]:
#helper method, executes a sql query and returns the fetchall results
def execute_query(query):
    cnx, c = connect_to_db()
    c.execute(query)
    result = c.fetchall()
    #close
    c.close()
    cnx.close()
    return result

In [4]:
def get_genres_from_db():
    cnx, cursor = connect_to_db()
    query = """SELECT * 
                FROM genres"""
    cursor.execute(query)
    return cursor.fetchall()

get_genres_from_db()

[('10402', 'Music'),
 ('10749', 'Romance'),
 ('10751', 'Family'),
 ('10752', 'War'),
 ('10770', 'TV Movie'),
 ('12', 'Adventure'),
 ('14', 'Fantasy'),
 ('16', 'Animation'),
 ('18', 'Drama'),
 ('27', 'Horror'),
 ('28', 'Action'),
 ('35', 'Comedy'),
 ('36', 'History'),
 ('37', 'Western'),
 ('53', 'Thriller'),
 ('80', 'Crime'),
 ('878', 'Science Fiction'),
 ('9648', 'Mystery'),
 ('99', 'Documentary')]

In [5]:

def convert_list_from_tuples(tuple_list):
    """The movie titles are retrieved as a list of tupes, 
    return instead a list with just the title from the first half of each tuple"""
    title_list = []
    for item in tuple_list:
        title_list.append(item[0])
    return title_list
    

def get_all_movie_info():
    select_q = """
                SELECT *
                FROM movies
                """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

def get_movie_names():
    name_q = """
            SELECT title
            FROM movies"""
    cnx, c = connect_to_db()
    c.execute(name_q)
    results = c.fetchall()
    cnx.close()
    c.close()
    title_list = convert_list_from_tuples(results) 
    return title_list

get_movie_names()
    

['Lock, Stock and Two Smoking Barrels',
 'Taxi Driver',
 'Perfect Blue',
 'Star Wars',
 'Wolf Children',
 'Chungking Express',
 'The Prestige',
 'Shutter Island',
 'Ran',
 'Sanjuro',
 'Yojimbo',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Lord of the Rings: The Two Towers',
 'The Lord of the Rings: The Return of the King',
 'Harry Potter and the Deathly Hallows: Part 2',
 'High and Low',
 'Spirited Away',
 'Forrest Gump',
 'Nostalgia',
 'Stalker',
 'Harakiri',
 'Ugetsu',
 'The Tale of the Princess Kaguya',
 'Citizen Kane',
 'The Dark Knight',
 'Interstellar',
 'Miracle in Cell No. 7',
 'A Clockwork Orange',
 'Dilwale Dulhania Le Jayenge',
 'The Red Shoes',
 'The Godfather',
 'Some Like It Hot',
 'The Godfather: Part II',
 'Il Sorpasso',
 'Big Deal on Madonna Street',
 'Nobody Knows',
 'Come and See',
 'Inception',
 'The Silence of the Lambs',
 'The Shawshank Redemption',
 'Love Exposure',
 'The Circus',
 'The Exterminating Angel',
 'Avengers: Infinity War',
 'Monsieur V

In [6]:
######SQL QUERIES#########

def get_most_popular_movie():
    q = """SELECT * 
            FROM movies
            ORDER BY popularity DESC
            LIMIT 5
        """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    cnx.close()
    c.close()
    return results

get_most_popular_movie()

def get_genre_counts():
    q = """SELECT genres.name AS genre_name, COUNT(*) 
        FROM genre_instances
        JOIN genres
        USING (genre_id)
        GROUP BY genre_instances.genre_id
    """
    cnx, c = connect_to_db()
    c.execute(q)
    results = c.fetchall()
    c.close()
    cnx.close()
    return results

get_genre_counts()

[('Music', 2),
 ('Romance', 17),
 ('Family', 7),
 ('War', 6),
 ('Adventure', 13),
 ('Fantasy', 15),
 ('Animation', 13),
 ('Drama', 71),
 ('Horror', 3),
 ('Action', 17),
 ('Comedy', 28),
 ('History', 8),
 ('Western', 2),
 ('Thriller', 14),
 ('Crime', 20),
 ('Science Fiction', 8),
 ('Mystery', 10)]

In [7]:
def get_most_popular_genres():
    q = """
        SELECT genres.name, AVG(popularity) AS avg_popularity
        FROM genre_instances
        JOIN genres 
        USING (genre_id)
        JOIN movies
        USING (movie_id)
        GROUP BY genre_id
        ORDER BY avg_popularity DESC;
        """
    return execute_query(q)

get_most_popular_genres()

[('Science Fiction', 48.87150001525879),
 ('Adventure', 40.18992299299974),
 ('Action', 32.61799986222211),
 ('Fantasy', 19.744733214378357),
 ('Thriller', 18.688071182795934),
 ('Family', 18.215285641806467),
 ('Crime', 17.60114974975586),
 ('Animation', 17.47823070562803),
 ('Western', 17.47000026702881),
 ('War', 16.40066687266032),
 ('Drama', 16.13723940244863),
 ('Comedy', 14.986321449279785),
 ('History', 14.899500131607056),
 ('Romance', 14.847823486608618),
 ('Mystery', 14.42209997177124),
 ('Horror', 10.439333279927572),
 ('Music', 8.884000301361084)]

In [25]:
def insert_second_data_to_table():
    pass